In [4]:
import pandas as pd

# Reading the first few rows of the CSV file
df = pd.read_csv('/Users/ron/Code/master_thesis/results/stats_2023-09-06 01:17:27.261880.csv')
df.head()


,Depth,Epoch,Steps,Task Name,Program,Examples
0,3,0,NaN,add-k with k=1,NaN,"[([[3, 4, 4, 13, 6]], [4, 5, 5, 14, 7]), ([[]]..."
1,3,1,NaN,pow-k with k=2,NaN,"[([[0]], [0]), ([[]], []), ([[]], []), ([[3]],..."
2,3,2,127.0,mult-k with k=4,(map (* 4) var0),"[([[3]], [12]), ([[]], []), ([[3, 5, 7, 4, 4, ..."
3,3,3,2.0,append-k with k=0,(append (length empty) var0),"[([[0, 14]], [0, 14, 0]), ([[4, 0]], [4, 0, 0]..."
4,3,4,413.0,modulo-k with k=2,(map (mod 2) var0),"[([[7, 0, 3, 12, 12]], [1, 0, 1, 0, 0]), ([[5,..."


In [8]:
# Count the number of non-NaN values in the 'Program' column for each depth
solved_programs_per_depth = df[df['Program'].notna()].groupby('Depth').size()
programs_per_depth =

print(solved_programs_per_depth)
print(programs_per_depth)


TypeError: 'int' object is not callable